In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
# from vgg16 import VGG16_LSTM
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import logging
import torch
import torch.nn as nn
from qx_noface_no2d_video_swin_transformer01 import SwinTransformer3D
import gc
from einops import rearrange
from tqdm import tqdm
# from kw_r2plus1d_video_swin_transformer import SwinTransformer3D
batchsz = 4
frame_nums=15
start_epoch = 0
test_flage=False
model_name ="13 "
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/personalityLog.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
USE_CUDA = torch.cuda.is_available()
# device = torch.device('cuda:1' if USE_CUDA else 'cpu')

In [2]:
logger.debug('============={}+Video swin transformerTrain Strat!============='.format(model_name))
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

[2024-04-05 12:02:00,448]::3435124833::DEBUG::=============13 +Video swin transformerTrain Strat!=============


True
학습을 진행하는 기기: cuda:0


In [3]:
def select_frame_data(datas,frame_num):
    data_set=[]
    begin_num=0
    x=128/frame_num
    indexes=[None] * frame_num
    for i in range(frame_num):
        indexes[i]=int(begin_num)
        begin_num+=x
    for data in datas:
        video_frames = [None] * 2
        video_frames[0]=[]
        video_frames[1]=[]
        for index in indexes:
            video_frames[0].append(data[0][index])
        video_frames[1].append(data[1])
        data_set.append(video_frames)
        del video_frames
        del data 
        gc.collect()
    del datas
    gc.collect()
    return data_set
print(1)

1


In [4]:
def getdata(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            # train_data_set.extend(pickle.load(output_file))
            final_data_set.extend(pickle.load(output_file))
            # print(i)
        return final_data_set
    elif datatype=="valid":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            valid_data_set=[]
            final_data_set.extend(pickle.load(output_file))
            del valid_data_set
        return final_data_set
print(1)

1


In [5]:
def datasize(type,batchsz):
    if type=='train':
        i=5076#5967
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
    else:
        i=1682#1986
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
print(1)

1


In [6]:

save_model_file_path = './save_swintransformer_folder/{}_{}.{}'
print(1)

1


In [7]:
def reshape_to_expected_traininput(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    result=[]
    for i in range(0, len(dataset[0])):
        result.append((dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i]))
    result = result[:datasize("train",batchsz)]
    x0_list = []
    x1_list = []
    x2_list = []
    x3_list = []
    for i in range(0, len(result)):
        x0_list.append(result[i][0])
        x1_list.append(result[i][1])
        x2_list.append(result[i][2])
        x3_list.append(result[i][3])
    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list),np.stack(x3_list))
print(1)

1


In [8]:
def reshape_to_expected_validinput(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    result=[]
    for i in range(0, len(dataset[0])):
        result.append((dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i]))
    result = result[:datasize("valid",batchsz)]
    x0_list = []
    x1_list = []
    x2_list = []
    x3_list = []
    for i in range(0, len(result)):
        x0_list.append(result[i][0])
        x1_list.append(result[i][1])
        x2_list.append(result[i][2])
        x3_list.append(result[i][3])
    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list),np.stack(x3_list))
print(1)

1


In [9]:
validation_set_data=getdata("/mnt/2ndSSD/qx/data/15Frames/margin/121314valid_set.dat","valid")
validation_set_data=reshape_to_expected_validinput(validation_set_data)
print(1)

1


In [10]:
train_set_data=getdata("/mnt/2ndSSD/qx/data/15Frames/margin/121314train_set.dat","train")
train_set_data=reshape_to_expected_traininput(train_set_data)
print(1)

1


In [11]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])
print(1)

1


In [12]:
class ChalearnDataset(Dataset):
    def __init__(self,fullshot,tagdata,transform=None):
        self.fullshot=fullshot
        self.tagdata=tagdata
        self.transform = transform  # 표준화 여부
    def __len__(self):
        return len(self.fullshot)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()#텐서의 경우 목록으로 돌아가기
        fullshot=self.fullshot[idx]
        fullshot=torch.FloatTensor(fullshot)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor(big_five_sorces)
        return fullshot,big_five_sorces
print(1)

1


In [13]:
num_workerssz = 20
lr = 3e-05
epochs = 120
print(1)

1


In [14]:
train_set_data = ChalearnDataset(fullshot = train_set_data[1],tagdata=train_set_data[3],transform=transform)
val_set_data = ChalearnDataset(fullshot = validation_set_data[1], tagdata=validation_set_data[3],transform=transform)
train_dataloader = DataLoader(dataset=train_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
val_dataloader = DataLoader(dataset=val_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
max_value=0
print(1)

1


In [15]:
model=SwinTransformer3D()#num_frames=frame_nums,batchsz = batchsz
model.to(device)
criterion = torch.nn.L1Loss().to(device)  # 손실함수
optimizer = torch.optim.AdamW(model.parameters(), lr=lr) # 옵티마이저

if test_flage==True:
    checkpoint=torch.load(save_model_file_path.format('model',start_epoch,'pth'), map_location=device)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    # criterion.load_state_dict(checkpoint["loss"])
train_avg_loss0=[]
val_avg_loss0=[]
print(1)

/home/ssrlab/anaconda3/envs/qx/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


1


In [16]:
# from torchinfo import summary

# summary(model, input_size = ((8,15,224,224,3),(8,24,1319,1)), col_names = ['input_size','output_size','num_params'])

In [17]:
def extract_ocean(data,batch_size):
    o=[]
    c=[]
    e=[]
    a=[]
    n=[]
    for i in range(batch_size):
        o.append(data[i][0].item())
        c.append(data[i][1].item())
        e.append(data[i][2].item())
        a.append(data[i][3].item())
        n.append(data[i][4].item())
    o = torch.tensor(o) 
    c = torch.tensor(c) 
    e = torch.tensor(e) 
    a = torch.tensor(a) 
    n = torch.tensor(n) 
    return o,c,e,a,n

In [18]:
with torch.cuda.device(0):
    for i in range(start_epoch, epochs):
        train_avg_loss = 0
        val_avg_loss = 0
        train_avg_o_loss = 0
        train_avg_c_loss = 0
        train_avg_e_loss = 0
        train_avg_a_loss = 0
        train_avg_n_loss = 0
        val_avg_o_loss = 0
        val_avg_c_loss = 0
        val_avg_e_loss = 0
        val_avg_a_loss = 0
        val_avg_n_loss = 0
        for fullshot,big_five_data in tqdm(train_dataloader):
            big_five_data=big_five_data.permute(0,2,1)
            big_five_data=big_five_data.squeeze()
            fullshot = fullshot.to(device)
            big_five_data = big_five_data.to(device)
            optimizer.zero_grad()  # 기울기가 0이 됩니다.
            hypothesis = model(fullshot)  # 모델의 예측 결과를 저장합니다.
            loss = criterion(hypothesis, big_five_data)  # 예측된 결과와 실제 태그 사이의 손실 값을 저장합니다.
            loss.backward()  # 역방향 전파입니다. 
            optimizer.step()  
            train_avg_loss += loss  
        train_avg_loss=train_avg_loss/len(train_dataloader)
        with torch.no_grad():#validate
            for fullshot,big_five_data in tqdm(val_dataloader):
                big_five_data=big_five_data.permute(0,2,1)
                big_five_data=big_five_data.squeeze()
                fullshot = fullshot.to(device)
                big_five_data = big_five_data.to(device)
                hypothesis = model(fullshot)
                val_loss = criterion(hypothesis, big_five_data)
                hypothesiso,hypothesisc,hypothesise,hypothesisa,hypothesisn=extract_ocean(hypothesis,batchsz)
                big_five_datao,big_five_datac,big_five_datae,big_five_dataa,big_five_datan=extract_ocean(big_five_data,batchsz)
                val_o_loss = criterion(hypothesiso, big_five_datao) 
                val_c_loss = criterion(hypothesisc, big_five_datac) 
                val_e_loss = criterion(hypothesise, big_five_datae) 
                val_a_loss = criterion(hypothesisa, big_five_dataa) 
                val_n_loss = criterion(hypothesisn, big_five_datan)  
                val_avg_loss += val_loss
                val_avg_o_loss += val_o_loss
                val_avg_c_loss += val_c_loss
                val_avg_e_loss += val_e_loss
                val_avg_a_loss += val_a_loss
                val_avg_n_loss += val_n_loss
            val_avg_loss=val_avg_loss/len(val_dataloader)
            val_avg_o_loss=val_avg_o_loss/len(val_dataloader)
            val_avg_c_loss=val_avg_c_loss/len(val_dataloader)
            val_avg_e_loss=val_avg_e_loss/len(val_dataloader)
            val_avg_a_loss=val_avg_a_loss/len(val_dataloader)
            val_avg_n_loss=val_avg_n_loss/len(val_dataloader)
        torch.cuda.empty_cache()
        # if (1-val_avg_loss)>=(1-max_value):
        max_value=val_avg_loss
        start_epoch+=1
        if (i + 1) % 5 == 0:
            torch.save({
                    'epoch': i+1,
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'loss': val_avg_loss,
                }, save_model_file_path.format('model',start_epoch,'pth'))
        logger.debug('Epoch: {} , 1-MAE: {:.4f} , 1-val_avg_o_loss:{:.4f} , 1-val_avg_c_loss:{:.4f} , 1-val_avg_e_loss:{:.4f} , 1-val_avg_a_loss:{:.4f} , 1-val_avg_n_loss:{:.4f}'.format(i+1, 1-val_avg_loss, 1-val_avg_o_loss, 1-val_avg_c_loss, 1-val_avg_e_loss, 1-val_avg_a_loss,1-val_avg_n_loss))
    

100%|██████████| 420/420 [00:31<00:00, 13.13it/s]
[2024-04-05 12:08:31,920]::3713155006::DEBUG::Epoch: 1 , 1-MAE: 0.8805 , 1-val_avg_o_loss:0.8841 , 1-val_avg_c_loss:0.8855 , 1-val_avg_e_loss:0.8868 , 1-val_avg_a_loss:0.8598 , 1-val_avg_n_loss:0.8865
100%|██████████| 420/420 [00:31<00:00, 13.13it/s]
[2024-04-05 12:13:24,335]::3713155006::DEBUG::Epoch: 2 , 1-MAE: 0.8901 , 1-val_avg_o_loss:0.8899 , 1-val_avg_c_loss:0.8922 , 1-val_avg_e_loss:0.8929 , 1-val_avg_a_loss:0.8816 , 1-val_avg_n_loss:0.8940
100%|██████████| 420/420 [00:32<00:00, 13.05it/s]
[2024-04-05 12:18:17,346]::3713155006::DEBUG::Epoch: 3 , 1-MAE: 0.8894 , 1-val_avg_o_loss:0.8948 , 1-val_avg_c_loss:0.8879 , 1-val_avg_e_loss:0.8981 , 1-val_avg_a_loss:0.8787 , 1-val_avg_n_loss:0.8877
100%|██████████| 420/420 [00:32<00:00, 13.00it/s]
[2024-04-05 12:23:10,676]::3713155006::DEBUG::Epoch: 4 , 1-MAE: 0.9008 , 1-val_avg_o_loss:0.9079 , 1-val_avg_c_loss:0.9003 , 1-val_avg_e_loss:0.9061 , 1-val_avg_a_loss:0.8852 , 1-val_avg_n_loss:0.9

In [19]:
# from torchinfo import summary

# summary(model, input_size = (4,15,4,3,224,224))